Following [this](https://docs.pydantic.dev/usage/postponed_annotations/)

**Read this first - what are postponed annotations and their problems: https://adamj.eu/tech/2021/05/15/python-type-hints-future-annotations/**

### Postponed annotations

> Both postponed annotations via the future import and `ForwardRef` require Python 3.7+.

They sort of "just works" most of the time, with pydantic.

In [1]:
from __future__ import annotations  # NOTE: This is the "opt in to postponed annotations" request.

from typing import Any, List
from pydantic import BaseModel


class Model(BaseModel):
    a: List[int]
    b: Any


print(Model(a=('1', 2, 3), b='ok'))  # pyright: ignore
#> a=[1, 2, 3] b='ok'


a=[1, 2, 3] b='ok'


* Internally, pydantic will call a method similar to `typing.get_type_hints` to resolve annotations.

In cases where the referenced type is not yet defined, `ForwardRef` can be used 
(although referencing the type directly or by its string is a simpler solution in the case of self-referencing models).

In some cases, a ForwardRef won't be able to be resolved during model creation. For example, this happens whenever a model references itself as a field type. When this happens, you'll need to call update_forward_refs after the model has been created before it can be used:

In [4]:
from typing import ForwardRef  # NOTE.
from pydantic import BaseModel

Foo = ForwardRef('Foo')  # pyright: ignore


class Foo(BaseModel):
    a: int = 123
    b: Foo = None  # pyright: ignore


Foo.update_forward_refs()  # NOTE: /!\ Big annoyance - this needs to be called...

print(Foo())
#> a=123 b=None
print(Foo(b={'a': '321'}))  # pyright: ignore
#> a=123 b=Foo(a=321, b=None)

a=123 b=None
a=123 b=Foo(a=321, b=None)


To resolve strings (type names) into annotations (types), pydantic needs a namespace dict in which to perform the lookup.
For this it uses `module.__dict__`, just like `get_type_hints`.

⚠️ This means pydantic may not play well with types not defined in the global scope of a module.

In [5]:
# For example, this works fine:

from __future__ import annotations
from pydantic import BaseModel
from pydantic import HttpUrl  # HttpUrl is defined in the module's global scope


def this_works():
    class Model(BaseModel):
        a: HttpUrl

    print(Model(a='https://example.com'))  # pyright: ignore
    #> a=HttpUrl('https://example.com', )


this_works()

a=HttpUrl('https://example.com', )


In [7]:
# While this will break:

from __future__ import annotations
from pydantic import BaseModel
from pydantic.errors import ConfigError


def this_is_broken():  # NOTE: /!\ Scope not being module level difference causes this to break...

    from pydantic import HttpUrl  # HttpUrl is defined in function local scope

    class Model(BaseModel):
        a: HttpUrl

    try:
        Model(a='https://example.com')  # pyright: ignore
    except ConfigError as e:
        print(e)
        #> field "a" not yet prepared so type is still a ForwardRef, you might
        #> need to call Model.update_forward_refs().

    try:
        Model.update_forward_refs()
    except NameError as e:
        print(e)
        #> name 'HttpUrl' is not defined


this_is_broken()

> ⚠️ Resolving this is beyond the call for pydantic: either remove the future import or declare the types globally.

### Self-referencing Models

Data structures with self-referencing models are also supported. Self-referencing fields will be automatically resolved after model creation.

Within the model, you can refer to the not-yet-constructed model using a string:

In [8]:
from pydantic import BaseModel


class Foo(BaseModel):
    a: int = 123
    #: The sibling of `Foo` is referenced by string
    sibling: 'Foo' = None  # pyright: ignore  # NOTE: this line.


print(Foo())
#> a=123 sibling=None
print(Foo(sibling={'a': '321'}))  # pyright: ignore
#> a=123 sibling=Foo(a=321, sibling=None)

a=123 sibling=None
a=123 sibling=Foo(a=321, b=None)


Since Python 3.7, you can also refer it by its type, provided you import `annotations`
(see above for support depending on Python and pydantic versions).

In [9]:
from __future__ import annotations  # NOTE.
from pydantic import BaseModel


class Foo(BaseModel):
    a: int = 123
    #: The sibling of `Foo` is referenced directly by type
    sibling: Foo = None  # pyright: ignore  # NOTE: this line.


print(Foo())
#> a=123 sibling=None
print(Foo(sibling={'a': '321'}))  # pyright: ignore
#> a=123 sibling=Foo(a=321, sibling=None)

a=123 sibling=None
a=123 sibling=Foo(a=321, sibling=None)
